In [ ]:
import pandas as pd
import operator as op
import numpy as np
from datetime import datetime
from numpy.random import rand
import operator as op
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
# load the data  
from google.colab import files
uploaded = files.upload()

In [ ]:
#Store the data
BitcoinPrice = pd.read_csv('Binance_BTCUSDT.csv', header = 1)
# # Add columns name
# BitcoinPrice.rename(columns={0: 'Date', 1: 'Price'}, inplace=True)
# BitcoinPrice.to_csv('BitcoinPrice.csv', index=False)
# Show the data
# BitcoinPrice.iloc[::-1]
#  BitcoinPrice.reset_index(inplace=True) # Resets the index, makes factor a column
BitcoinPrice.iloc[::-1].to_csv('Bitcoin.csv',index=False)
BitcoinPrice = pd.read_csv('Bitcoin.csv')
BitcoinPrice

In [ ]:
def graph():
  # Visualize the data
  plt.figure(figsize=(31, 15))
  plt.plot(BitcoinPrice['Open'],label = 'Bitcoin Price')
  plt.title('Bitcoin price history')
  plt.xlabel('Aug. 17, 2017 - Aug. 08, 2020')
  plt.ylabel('USD($)')
  plt.legend(loc='upper left')
  return(plt.show())
graph()

In [ ]:
# Create a new data frame with a simple moving 10-300 day average to store all the data
def create_sma():
  sma = create_dic(30)
  data = pd.DataFrame()
  data['Bitcoin Price'] = BitcoinPrice['Open']
  for key in sma:
    data[key] = BitcoinPrice['Open'].rolling(window=sma[key]).mean()
  return(data)
  
def create_dic(n):
  SMA ={}
  for i in range(1,n+1):
    sma = 'SMA'
    sma += str(i*10)
    SMA.update( {sma : i*10} )
  return(SMA)

create_sma()

In [ ]:
def graph_sma(): 
  # Visualize the data
  data = create_sma()
  plt.figure(figsize=(31,15))
  plt.plot(data['Bitcoin Price'],label = 'Bitcoin Price')
  sma = create_dic(30)
  number_of_plots = 30
  for key in sma:
    rgb = np.random.rand(3,)
    plt.plot(data[key], label = key, c=rgb, linewidth=3)
  plt.title('Bitcoin price history and SMA')
  plt.xlabel('Aug. 17, 2017 - Aug. 08, 2020')
  plt.ylabel('USD($)')
  plt.legend(loc='upper left')
  return(plt.show())
graph_sma()

In [ ]:
def signals(sma1,sma2):
  data = create_sma()
  sigPriceBuy = []
  sigPriceSell = []
  flag = -1
  for i in range(len(data)):
    # if data[sma1].head(1) > data[sma2].head(1) and data[sma1].tail(1) < data[sma2].tail(1):
    if data[sma1][i] > data[sma2][i]:
      if flag != 1:
        sigPriceBuy.append(data['Bitcoin Price'][i])
        sigPriceSell.append(np.nan)
        flag = 1
      else:
        sigPriceBuy.append(np.nan)
        sigPriceSell.append(np.nan)
    elif data[sma1][i] < data[sma2][i]:
      if flag != 0:
        sigPriceBuy.append(np.nan)
        sigPriceSell.append(data['Bitcoin Price'][i])
        flag = 0
      else:
        sigPriceBuy.append(np.nan)
        sigPriceSell.append(np.nan)
    else:
      sigPriceBuy.append(np.nan)
      sigPriceSell.append(np.nan)
  return data, sigPriceBuy, sigPriceSell


In [ ]:
def add_Buy_Sell(data,sma1,sma2):
  data, data['Buy_signal_price'], data['Sell_signal_price']= signals(sma1,sma2)
  # Droping Nan in two frames
  df = pd.DataFrame(data['Buy_signal_price']) 
  dfBuy = df.dropna()
  df2 = pd.DataFrame(data['Sell_signal_price']) 
  dfSell= df2.dropna()
  b =[]
  s = []
  for row in dfBuy.index: 
    b.append(row) 
  for row in dfSell.index: 
    s.append(row) 
  bh = b[0]
  sh = s[0]
   
  bt = b[len(b)-1]
  st = s[len(s)-1]
  # We drop the first sell operation and last buy operation
  if sh < bh:
    dfSell = dfSell.drop(sh)
  if st < bt:
    dfBuy = dfBuy.drop(bt)
  data['Buy_signal_price'], data['Sell_signal_price'] = dfBuy, dfSell
   # Concatenate the frames
  Buy_Sell = pd.concat([dfBuy,dfSell])
  # Sorting data
  dataBS = Buy_Sell.sort_index() 

  return data, dfBuy, dfSell, dataBS

In [ ]:
def algoprofit(sma1,sma2): 
  # Store the buy and sell data into a variable
  # data, data['Buy_signal_price'], data['Sell_signal_price']= signals(sma1,sma2)
  data, df, df2, dBS = add_Buy_Sell(create_sma(),sma1, sma2) 
  buy = df['Buy_signal_price'].sum()
  sell = df2['Sell_signal_price'].sum()
  result = sell - buy
  return(result)  

In [ ]:
def profit(sma1,sma2):
  data, df, df2, dBS = add_Buy_Sell(create_sma(),sma1, sma2) 
  a = df['Buy_signal_price'].values[0]
  b = df2['Sell_signal_price'].values[len(df2['Sell_signal_price']) - 1]
  result = b - a 
  return result

In [ ]:
# Create a function to signal wen to buy and sell the asset
def graph_smas(sma1,sma2):
   #  Visualize the signals
  print('\n____________________________________________________________________________________________________________________________________________\n\n\t\t\t\tWith the paar: ', sma1, ' and ', sma2, ' we have next buy-sell signals in the graph below: \n\t')
  data, df, df2, dBS = add_Buy_Sell(create_sma(),sma1, sma2) 
  plt.figure(figsize=(32, 15))
  plt.plot(data['Bitcoin Price'], label = 'Bitcoin Price', alpha = 0.35)
  plt.plot(data[sma1], label = sma1, alpha = 0.35)
  plt.plot(data[sma2], label = sma2, alpha = 0.35)
  plt.scatter(data.index, data['Buy_signal_price'], label = 'Buy', marker = '^', color = 'green')
  plt.scatter(data.index, data['Sell_signal_price'], label = 'Sell', marker = 'v', color = 'red')
  plt.title('Bitcoin_Buy_Sell_Signals')
  plt.xlabel('Aug. 17, 2017 - Aug. 12, 2020')
  plt.ylabel('USD($)')
  plt.legend(loc='upper left')
  plt.show()
  print('\n\tHERE THE BUY-SELL TABLE :\n\n\t',dBS)
  text = 'THE PROFIT WITH'
  print('\n\n\t', text, sma1, ' AND ', sma2,' IS :', algoprofit(sma1,sma2))
  print('\n\tWithout this algorythm the profit would be only :', profit(sma1,sma2))



In [ ]:
def main():
  data = create_dic(30)
  sig = []
  prof_sma = {}
  for key in data:
    sig.append(key)
  for i in range(len(sig)):
    for j in range(len(sig)):  
      if j > i and algoprofit(sig[i],sig[j]) > 0:
        sign = sig[i]+sig[j]
        val= algoprofit(sig[i],sig[j])
        prof_sma.update({sign: val})
  sorted_d = dict(sorted(prof_sma.items(), key=op.itemgetter(1),reverse=True))
  print('\n\t\t\t\tNEXT 5 SMAs CROSSING PRICE WILL GIVE US MAXIMUM PROFIT \n')
  print('\n\tIn the graphs below we will illustate these signals :\n')

  for a in list(sorted_d)[0:5]:
    sig1 = ''
    sig2 = ''
    if len(a)<=10:
      for k in list(a)[0:5]:
        sig1+=k
      for k in list(a)[5:10]:
        sig2+=k
    elif len(a) == 11:
      for k in list(a)[0:5]:
        sig1+=k
      for k in list(a)[5:11]:
        sig2+=k
    else:
      for k in list(a)[0:6]:
        sig1+=k
      for k in list(a)[6:12]:
        sig2+=k
    if a == 0:
      print('THE MAXIMUM PROFIT WITH PRICE CROSSING OF ', sig1, 'AND' , sig2, ' :', str(sorted_d[a]).upper())
    graph_smas(sig1,sig2)

main()